## 00 - Preparamos el entorno cargando las bibliotecas necesarias
En caso de que dichas bibliotecas no estén instaladas, es necesario ejecutar el siguiente bloque de código
```
! pip install --upgrade requests selenium BeautifulSoup pandas
```

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

## 01 - Por qué usaremos un navegador emulado?

A diferencia de lo que hicimos con [transparenciapresupuestaria.gob.mx](https://www.transparenciapresupuestaria.gob.mx/) usando requests, la información que obtendremos de [rhnet.gob.mx](https://www.rhnet.gob.mx) requiere una aproximación muy diferente

rhnet.gob.mx cuenta con una API, sin embargo requiere de información adicional que no es tan fácil de automatizar (spoiler: si es ;) )

### 01.0.1 Espacio para mostrar el uso fallido de la API de rhnet.gob.mx

En la demostración vemos que en efecto hay una llamada a la API de RHNet a través de la dirección `https://www.rhnet.gob.mx/servlet/CheckSecurity/JSP/mss_g1/mss_g1_organigrama_pos1_xls.jsp`, no obstante, no podemos usar `requests` o `curl` ya que el **header** que la api requiere para su buen funcionamiento incluye cookies, un referente, y un certificado.

Cómo podemos obtener de forma automática dichos datos?

### 01.1 - Opciones de nuestro navegador emulado

En la siguiente celda crearemos un objeto llamado `options` que contiene argumentos *invisibles* que usamos de forma cotidiana en nuestro navegador.

Las opciones que le vamos a pasar al webdriver de chrome garantizan que podamos automatizar el código sin tener que abrir un navegador cada vez.

Por qué crees que el argumento `--headless` está desactivado?

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--disable-extensions")
#options.add_argument("--headless")

### 01.2 - Inicializando el navegador emulado

Ya que construimos la serie de argumentos que nuestro navegador emulado requiere, podemos inicializarlo ejecutando el código de la siguiente celda

In [ ]:
browser = webdriver.Chrome(options=options)

### 01.3 - Abrimos la página web de RHNet

In [ ]:
browser.get("https://www.rhnet.gob.mx/servlet/CheckSecurity/JSP/mss_g1/mss_g1_organigrama_pos_filtro.jsp")

## 02 - Elementos emergentes, y como atraparlos en nuestro navegador emulado

La primera dificultad que encontramos en la automatización es la detección de elementos emergentes

Cada sitio web es diferente, y no hay un `class` universal para detectar los elementos emergentes

Podemos usar XPATH para seleccionar el boton de cerrar en el elemento emergente y hacer click en dicho boton para actualizar el DOM a la página que queremos visitar

In [ ]:
popup_link = browser.find_element(By.XPATH,"/html/body/div[2]/div/div[1]/button")
popup_link.click();

## 03 - Cambiando de página, un click a la vez

Una vez que nos deshicimos del elemento emergente, podemos pasar a la sección de búsqueda

A diferencia del elemento emergente, el link de búsqueda si tiene un `id` específico, por lo que facilmente podemos hacer click en el link ejecutando el siguiente código

In [ ]:
search_link = browser.find_element(by="id",value="linkEOR")

In [ ]:
search_link.click();

## 04 - Por qué usaremos un navegador emulado? Si, de nuevo

Con selenium podemos hacer automatización de procesos de una forma elegante, si acaso rebuscada. En la siguiente sesión se cubrirá un uso más detallado de selenium

Para esta sesión, en realidad estamos usando selenium para obtener los cookies que ocupamos para *engañar* a la API de RHNet y hacerle creer que es un humano quien está tratando de obtener información de su portal

In [ ]:
cookie_list = browser.get_cookies()

### 04.1 - Explorando las cookies obtenidas

Selenium nos entrega un diccionario de cookies, indicando el nombre y el valor de cada cookie.

In [ ]:
cookie_list

### 04.2 - Construyendo el encabezado para la API, una cookie a la vez

Al ejecutar el siguiente código, podemos transformar el objeto `cookie_list` en un `string` que podemos incluir en el encabezado para la solicitud en la API

In [ ]:
cookie_str = "\nCookie: "
for i in range(len(cookie_list)):
    cookie_name = cookie_list[i]["name"]
    cookie_value = cookie_list[i]["value"]
    cookie_str += cookie_name +"="+cookie_value+"; "

In [ ]:
print(cookie_str)

## 05 - Estableciendo una conexión segura

El diseño de la API de RHNet es interesante ya que sigue buenas prácticas de seguridad, autenticación y geolocalización.
>El protocólo de transferencia de datos deja mucho que desear, no se puede tener todo en la vida

Antes de realizar cualquier llamada, necesitamos obtener un certificado de seguridad que garantice la seguridad de la conexión
>Cada página web ***segura*** hace uso de certificados que protegen la transparencia de los datos entre el servidor y el cliente, pero proteje de intermediarios externos.

Usaremos un programa llamado [OpenSSL](https://www.openssl.org/) el cual nos permite obetener y autenticar certificados

>Sólo debes ejecutar una de las siguientes celdas, elige la correspondiente a tu sistema operativo

>Es importante que en caso de utilizar windows, ajustes la ruta del programa openssl.exe

In [ ]:
# Unix/Linux/Mac
! openssl s_client -showcerts -connect www.rhnet.gob.mx:443 </dev/null 2>/dev/null | openssl x509 -outform PEM > certificado.pem

In [ ]:
# Windows
! openssl.exe s_client -showcerts -connect www.rhnet.gob.mx:443 < nul 2> nul | openssl.exe x509 -outform PEM > certificado.pem

## 06 - Construyendo el encabezado

Ya que la primera parte del encabezado es esencialmente fija, podemos usar una [plantilla](headers.template) y sobre dicha plantilla agregar las cookies

Al ejecutar el siguiente código, crearemos un nuevo archivo llamado [headers](headers), que contiene la primera parte de la plantilla, y agregamos la sección correspondiente a las cookies

In [ ]:
with open('headers.template','r') as template, open('headers','w') as target: 
    for line in template:
        target.write(line)
with open("headers", "a") as target:
    target.write(cookie_str)

## 07 - Preparando la solicitud

En la demostración del sitio web de RHNet, vimos que la API requiere parámetros específicos, con el siguiente código podemos guardar dichos parámetros en un archivo de texto plano

In [ ]:
data = "ID_ORGANIZATION=02&ORGANO=todos&FEC=14-10-2024"
with open("data", "w") as target:
    target.write(data)

## 08 - Drumroll, la solicitud

... usando curl en vez de requests

Requests requiere dos tipos de certificado para la autenticación de la API de RHNet (puntos extra por la seguridad), no obstante, `curl`, solamente requiere un certificado, el que obtuvimos con `openssl`

Ya construimos el encabezado, ya tenemos el certificado y ya tenemos la data para la API.

Con el siguiente código obtendremos la información de la API de RHNet

>Puedes identificar algo raro en la información obtenida?

>Qué red flag tiene la API de RHNet en cuanto a transferencia de información?

In [ ]:
! curl --request POST --cacert certificado.pem --header @headers --data @data "https://www.rhnet.gob.mx/servlet/CheckSecurity/JSP/mss_g1/mss_g1_organigrama_pos1_xls.jsp" > results.html 2> curl.err

## 09 - Con ***Ñ*** porque estamos en México

> Qué pasa si haces doble click en el documento [results.html](results.html) que se generó con curl?

> Qué pasa si intentas cargarlo a beautifulsoup?

In [ ]:
with open("results.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

### 09.1 de UTF y otras [codificaciones](https://es.wikipedia.org/wiki/Codificaci%C3%B3n_de_caracteres)

Para bien y para mal, la API de RHNet manda los resultados en una codificación que beautifulsoup no entiende, por lo anterior, debemos cambiar la codificación del documento html a algo más sencillo, [UTF-8](https://es.wikipedia.org/wiki/UTF-8)

Con el siguiente código crearemos un nuevo documento html con codificación UTF-8

https://es.wikipedia.org/wiki/UTF-8

>Sólo debes ejecutar una de las siguientes celdas, elige la correspondiente a tu sistema operativo

>Es importante que en caso de utilizar windows, ajustes la ruta del programa iconv.exe

In [ ]:
# Unix/Linux/Mac
! iconv -f iso-8859-1 -t utf-8 -o results.clean.html results.html

In [ ]:
# windows
! iconv.exe -f iso-8859-1 -t utf-8 -o results.clean.html results.html

## 10 - Parseando el resultado con bs4

Ya que tenemos nuestro documento en UTF-8 podemos leerlo con beautifulsoup y ahora si podemos hacer manejo de datos para obtener una tabla más manejable

In [ ]:
with open("results.clean.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

### 10.1 - Tablas dentro de divs, qué buscar y donde

Una vez que exploramos el contenido de nuestro elemento `soup`, podemos ver que hay cierta estructura.

Las tablas que nos interesan están dentro de elementos genéricos de tipo `<div>`, los cuales tienen `id`s específicos

Podemos pasarle a bs4 dichos `id`s y continuar con la extracción de datos

In [ ]:
tipo_personal_table = soup.find('div', id='ResTipoPersonal')
car_ocupacion_table = soup.find('div', id='ResCarOcu')
tipo_funcion_table  = soup.find('div', id='ResTipoFun')
puestos_table       = soup.find('div', id='RepPuestos')

### 10.2 Resumen de tipo de personal

Con el siguiente código podemos obtener una lista que contiene diccionarios anidados con la información correspondiente al tipo de personal

In [ ]:
tipo_personal_list = []
for row in range(2,len(tipo_personal_table.find_all("tr"))-1):
    col_dict = {}
    cols    = tipo_personal_table.find_all("tr")[row].find_all("td")
    tipo    = cols[0].text.rstrip(' ').lstrip(' ')
    plazas  = cols[1].text.rstrip(' ').lstrip(' ')
    porcent = cols[2].text.rstrip(' ').lstrip(' ')
    col_dict["tipo"]    = tipo
    col_dict["plazas"]  = plazas
    col_dict["porcent"] = porcent
    tipo_personal_list.append(col_dict)

#### 10.2.1 Y podemos convertir dicha lista en un dataframe de pandas

In [ ]:
tipo_personal_df = pd.DataFrame.from_records(tipo_personal_list)

In [ ]:
tipo_personal_df

### 10.3 Resumen de característica ocupacional

Realizamos el mismo procedimiento con la tabla de características ocupacionales

In [ ]:
car_ocupacion_list = []
for row in range(2,len(car_ocupacion_table.find_all("tr"))-1):
    col_dict = {}
    cols    = car_ocupacion_table.find_all("tr")[row].find_all("td")
    caract  = cols[0].text.rstrip(' ').lstrip(' ')
    plazas  = cols[1].text.rstrip(' ').lstrip(' ')
    porcent = cols[2].text.rstrip(' ').lstrip(' ')
    col_dict["caracteristica"] = caract
    col_dict["plazas"]  = plazas
    col_dict["porcent"] = porcent
    car_ocupacion_list.append(col_dict)

#### 10.3.1 Lo pasamos a un dataframe

In [ ]:
car_ocupacion_df = pd.DataFrame.from_records(car_ocupacion_list)

In [ ]:
car_ocupacion_df

### 10.4 Resumen de tipo de funciones

Realizamos el mismo procedimiento con la tabla de tipo de funciones

In [ ]:
tipo_funcion_list = []
for row in range(2,len(tipo_funcion_table.find_all("tr"))-2):
    col_dict = {}
    cols    = tipo_funcion_table.find_all("tr")[row].find_all("td")
    funcion = cols[0].text.rstrip(' ').lstrip(' ')
    plazas  = cols[1].text.rstrip(' ').lstrip(' ')
    porcent = cols[2].text.rstrip(' ').lstrip(' ')
    col_dict["función"] = funcion
    col_dict["plazas"]  = plazas
    col_dict["porcent"] = porcent
    tipo_funcion_list.append(col_dict)

#### 10.4.1 Lo pasamos a un dataframe

In [ ]:
tipo_funcion_df = pd.DataFrame.from_records(tipo_funcion_list)

In [ ]:
tipo_funcion_df

### 10.5 Cosas que no se ven pero que ahí están

Esta última parte no se muestra en el navegador, sin embargo curl si logró captar el contenido de la tabla que se descarga tras realizar la búsqueda

#### 10.5.1 `if(len(cols)>=3): use for loop`

Con el siguiente código podemos obtener los nombres de las columnas de nuestra tabla, los cuales almacenaremos en una lista

In [ ]:
puestos_header = puestos_table.find_all("tr")[1].find_all("th")

In [ ]:
puestos_cols = []
for col_num in range(len(puestos_header)):
    col_name = puestos_header[col_num].text.rstrip(' ').lstrip(' ')
    puestos_cols.append(col_name)

In [ ]:
puestos_cols

#### 10.5.2 La tabla que nos interesa

Aplicaremos la misma mecánica que hemos usado anteriormente, sin embargo, a la hora de construir los diccionarios, usaremos un loop anidado para asignar automaticamente las etiquetas de cada diccionario

In [ ]:
puestos_list = []
for row in range(2,len(puestos_table.find_all("tr"))):
    col_dict = {}
    cols    = puestos_table.find_all("tr")[row].find_all("td")
    for col in range(len(puestos_cols)):
        label = puestos_cols[col]
        value = cols[col].text.rstrip(' ').lstrip(' ')
        col_dict[label] = value
    puestos_list.append(col_dict)

#### 10.5.3 La pasamos a un dataframe de pandas

In [ ]:
puestos_df = pd.DataFrame.from_records(puestos_list)

In [ ]:
puestos_df

#### 10.5.4 Y almacenamos nuestros resultados en una archivo `.tsv`

In [ ]:
puestos_df.to_csv("estructuras.tsv",sep="\t",index=False)